###### Build a movie recommender system using a Python library such as Surprise or FastFM.

In [44]:
# import libraries
import pandas as pd
import os, io
import numpy as np
import surprise
from pandas import Series, DataFrame, read_table
from surprise import Dataset
from surprise.model_selection import KFold ,cross_validate
from surprise import KNNBasic, Reader,Dataset,SVD
from surprise.model_selection import cross_validate
from surprise.accuracy import rmse
from surprise.model_selection import train_test_split

In [9]:
# load dataset
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file('./data/ml-100k/u.data', reader=reader,)


In [10]:
kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x7fb5e9599e40>

In [11]:
algo = KNNBasic(sim_options={'user_based':False}) 

In [12]:
cross_validate(algo,data,measures = ['rmse','mae'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.97205826, 0.96769216, 0.97699906, 0.98021216, 0.9707333 ]),
 'test_mae': array([0.76764394, 0.76623195, 0.77252127, 0.77316184, 0.76626819]),
 'fit_time': (0.7465410232543945,
  0.7235636711120605,
  0.6627159118652344,
  0.680964469909668,
  0.7012472152709961),
 'test_time': (5.2393341064453125,
  4.93437647819519,
  4.64732027053833,
  4.61664891242981,
  4.581440687179565)}

In [37]:
trainset = data.build_full_trainset()
testSet = trainset.build_anti_testset()

In [38]:

train, test = train_test_split(data, test_size=0.25)
predictions = algo.test(test)


In [39]:
from collections import defaultdict

def get_top3_recommendations(predictions, topN = 3):

    top_recs = defaultdict(list)

    for uid, iid, true_r, est, _ in predictions:

        top_recs[uid].append((iid, est))
    for uid, user_ratings in top_recs.items():

        user_ratings.sort(key = lambda x: x[1], reverse = True)

        top_recs[uid] = user_ratings[:topN]
    return top_recs


In [40]:
def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and returns a
    mapping to convert raw ids into movie names.
    """
    file_name = ('./data/ml-100k//u.item')
    rid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
    return rid_to_name

In [41]:
top3_recommendations = get_top3_recommendations(predictions)
rid_to_name = read_item_names()
recommendations = list()
for uid, user_ratings in top3_recommendations.items():
    reco = [uid, [rid_to_name[iid] for (iid, _) in user_ratings]]
    recommendations.append(reco)
    #print(uid, [rid_to_name[iid] for (iid, _) in user_ratings])

df = DataFrame(recommendations,columns=['u_id','movie title'])

In [42]:
df[df['u_id']=='13']['movie title'].values

array([list(['Dangerous Beauty (1998)', 'Kundun (1997)', 'Wyatt Earp (1994)'])],
      dtype=object)

In [45]:
accuracy = surprise.accuracy.rmse(predictions)

RMSE: 0.7628


In [46]:
accuracy

0.7628005377513208